In [2]:
!pip install requests

     |████████████████████████████████| 62 kB 866 kB/s eta 0:00:01
     |████████████████████████████████| 138 kB 1.7 MB/s eta 0:00:01
     |████████████████████████████████| 59 kB 5.8 MB/s eta 0:00:011


In [4]:
!pip install xmltodict

### 데이터 수집 crawl_emergency.py & crawl_emergency.sh 참고  
test 부분

In [19]:
# import pymysql
import requests
# from bs4 import BeautifulSoup
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import time
import os
import json
import datetime
import xmltodict

s = requests.Session()
retries = Retry(total=5,
               backoff_factor=4, # 2, 4, 8, 16, 32
               status_forcelist=[500, 502, 503, 504])
headers= {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:87.0) Gecko/20100101 Firefox/87.0'
}

def crawl(base_url, params, directory, name):
    today = datetime.datetime.now()
    res = s.get(base_url, headers=headers, params=params)
    if res.status_code == 200:
        result = json.loads(json.dumps(xmltodict.parse(res.content.decode('utf-8')), ensure_ascii=False))
        with open(directory + today.strftime("%Y%m%d%H%M%S") + name + ".json", "w", encoding='utf-8') as json_file:
            json.dump(result, json_file, ensure_ascii=False)
        return result
    else:
        return res.status_code

key = [
    'P/A790R+948U9aRNFwyvH4s4J5rakjk7BJbSNZxKlDoqr9XI1kgbPO+CoH9EwJsJG8Ex4TaortyjJGid58aBAw==',
]

# cron2 weather
# 1시간 간격 or 하루 마지막에 한번만
today = datetime.datetime.now()
base_url = 'http://apis.data.go.kr/1741000/DisasterMsg3/getDisasterMsg1List'
params = {
            'ServiceKey': key,
            'pageNo': 1,
            'numOfRows': 100,
            'dataType': 'XML',
            'base_date': today.strftime("%Y%m%d"),
            'base_time': (today - datetime.timedelta(hours=1)).strftime("%H00"),
        }

crawl(base_url, params, './emergency-test/', 'EmergencyMsg'+ today.strftime("%Y%m%d"))


{'DisasterMsg': {'head': {'list_total_count': '100824',
   'RESULT': {'CODE': 'INFO-0', 'MESSAGE': '정상'}},
  'row': [{'create_date': '2021/09/07 14:39:26',
    'location_id': '74',
    'location_name': '경상북도 전체',
    'md101_sn': '121317',
    'msg': '[경상북도청] 18~49세 예방접종 대상자는 콜센터(☎1339 및 보건소) 또는 누리집(https://ncvr2.kdca.go.kr)에서 사전예약하시기 바랍니다.',
    'send_platform': 'cbs'},
   {'create_date': '2021/09/07 14:38:53',
    'location_id': '54',
    'location_name': '경상남도 거제시',
    'md101_sn': '121316',
    'msg': '[거제시청] 9/4(토) 11:10 고현버스터미널 출발→부산사상터미널 도착 버스이용자는 가까운 선별진료소에서 코로나검사바랍니다.',
    'send_platform': 'cbs'},
   {'create_date': '2021/09/07 14:38:36',
    'location_id': '54',
    'location_name': '경상남도 거제시',
    'md101_sn': '121315',
    'msg': '[거제시청] 9/1(수) 13:35 고현버스터미널 출발→서울남부터미널 도착 버스이용자는 가까운 선별진료소에서 코로나검사바랍니다.',
    'send_platform': 'cbs'},
   {'create_date': '2021/09/07 14:33:30',
    'location_id': '28',
    'location_name': '경기도 군포시',
    'md101_sn': '121314',
    'msg': '[군포시청] 9

### 전처리

In [4]:
import sys
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode

In [5]:
spark = SparkSession.builder.appName('dataframe').getOrCreate()

In [6]:
msg_all = spark.read.json("/home/lab11/emergency/*.json")
msg_all.printSchema()
msg_all.show()
msg_all.show(truncate=False)

root
 |-- DisasterMsg: struct (nullable = true)
 |    |-- head: struct (nullable = true)
 |    |    |-- RESULT: struct (nullable = true)
 |    |    |    |-- CODE: string (nullable = true)
 |    |    |    |-- MESSAGE: string (nullable = true)
 |    |    |-- list_total_count: string (nullable = true)
 |    |-- row: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- create_date: string (nullable = true)
 |    |    |    |-- location_id: string (nullable = true)
 |    |    |    |-- location_name: string (nullable = true)
 |    |    |    |-- md101_sn: string (nullable = true)
 |    |    |    |-- msg: string (nullable = true)
 |    |    |    |-- send_platform: string (nullable = true)
 |-- OpenAPI_ServiceResponse: struct (nullable = true)
 |    |-- cmmMsgHeader: struct (nullable = true)
 |    |    |-- errMsg: string (nullable = true)
 |    |    |-- returnAuthMsg: string (nullable = true)
 |    |    |-- returnReasonCode: string (nullable = true)
 |

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
msg_all.select('DisasterMsg.row').show()

+--------------------+
|                 row|
+--------------------+
|[{2021/09/07 14:3...|
|                null|
|                null|
|[{2021/09/07 11:0...|
|[{2021/09/03 11:0...|
|[{2021/09/03 11:0...|
|[{2021/09/03 11:0...|
|[{2021/09/03 11:0...|
|[{2021/09/06 15:0...|
|[{2021/09/06 15:0...|
|[{2021/09/03 14:4...|
|[{2021/09/03 14:4...|
|[{2021/09/03 14:4...|
|[{2021/09/03 14:4...|
|[{2021/09/03 14:4...|
|[{2021/09/07 14:3...|
|[{2021/09/07 14:3...|
|[{2021/09/07 14:3...|
|[{2021/09/07 14:3...|
|[{2021/09/07 14:3...|
+--------------------+
only showing top 20 rows



In [8]:
msg_all.select('DisasterMsg.row').printSchema()

root
 |-- row: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- create_date: string (nullable = true)
 |    |    |-- location_id: string (nullable = true)
 |    |    |-- location_name: string (nullable = true)
 |    |    |-- md101_sn: string (nullable = true)
 |    |    |-- msg: string (nullable = true)
 |    |    |-- send_platform: string (nullable = true)



In [9]:
msg_all.select(explode(msg_all.DisasterMsg.row)).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- create_date: string (nullable = true)
 |    |-- location_id: string (nullable = true)
 |    |-- location_name: string (nullable = true)
 |    |-- md101_sn: string (nullable = true)
 |    |-- msg: string (nullable = true)
 |    |-- send_platform: string (nullable = true)



In [10]:
df_msg = msg_all.select(explode(msg_all.DisasterMsg.row).alias("msg")).select('msg.*') # col을 msg로 변경 후 msg만 선택

In [11]:
df_msg = df_msg.toPandas()

In [12]:
df_msg

,create_date,location_id,location_name,md101_sn,msg,send_platform
0,2021/09/07 14:39:26,74,경상북도 전체,121317,[경상북도청] 18~49세 예방접종 대상자는 콜센터(☎1339 및 보건소) 또는 누...,cbs
1,2021/09/07 14:38:53,54,경상남도 거제시,121316,[거제시청] 9/4(토) 11:10 고현버스터미널 출발→부산사상터미널 도착 버스이용...,cbs
2,2021/09/07 14:38:36,54,경상남도 거제시,121315,[거제시청] 9/1(수) 13:35 고현버스터미널 출발→서울남부터미널 도착 버스이용...,cbs
3,2021/09/07 14:33:30,28,경기도 군포시,121314,[군포시청] 9월 7일 14시 기준 코로나19 확진자 16명 발생. 상세내용은 군포...,cbs
4,2021/09/07 14:30:56,25,경기도 광명시,121313,[광명시청] 8.30.(월) ~ 9.4.(토) 광명동 소재 서울내과의원(광명시 광명...,cbs
...,...,...,...,...,...,...
2795,2021/09/02 10:11:24,200,전라남도 해남군,120602,[해남군청]타지역확진자 관내 동선안내▶8.21.16:00~16:30 땅끝농협하나로마...,cbs
2796,2021/09/02 10:10:44,71,경상남도 함안군,120601,"[함안군청] 9월 2일 확진자 16명(함안209~224, 확진자접촉16) 발생, 마...",cbs
2797,2021/09/02 10:05:50,167,울산광역시 중구,120600,[중구청] 임시선별검사소▲성남둔치 주차장(평일9시~17시) ▲울산종합운동장(9시~1...,cbs
2798,2021/09/02 10:05:10,71,경상남도 함안군,120599,[함안군청] 8.23(월)~8.31(화) 우리집막창(법수면 소재) 방문자는 근처 선...,cbs


In [ ]:
# py 파일 내용 - 7200초 마다 파일을 하나로 합침.

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode

spark = SparkSession \
    .builder \
    .appName("emergencyMsg") \
    .getOrCreate()

MsgSchema = spark.read.format('json').load('/home/lab11/emergency/20210903105648EmergencyMsg20210903.json').schema
MsgDf = spark.readStream.schema(MsgSchema).json('/home/lab11/emergency/*.json')
df_msg = MsgDf.select(explode(MsgDf.DisasterMsg.row).alias("emergency")).select('emergency.*')
df_msg.coalesce(1).writeStream.format('json') \
    .option("checkpointLocation", "/home/lab11/emergency_check") \
    .option("path", "/home/lab11/emergency_msg") \
    .trigger(processingTime='7200 seconds') \
    .start().awaitTermination()

KeyboardInterrupt: 

### process.py와 process.sh 실행 이후는 아래 방법으로 불러오기 가능

In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode

spark = SparkSession.builder.appName('dataframe').getOrCreate()

df_msg = spark.read.json("/home/lab11/emergency_msg/*.json")
df_msg = df_msg.toPandas()

In [15]:
df_msg

,create_date,location_id,location_name,md101_sn,msg,send_platform
0,2021/09/07 11:00:05,229,충청남도 서산시,121287,"[서산시청] 9/7 11시 기준 확진자 7명 발생(634~640/격리중6, 관내접촉...",cbs
1,2021/09/07 11:00:04,33,경기도 성남시,121286,"[성남시청] 9월 7일 0시 기준 신규확진자 67명(수정구 14, 중원구 10, 분...",cbs
2,2021/09/07 10:59:52,33,경기도 성남시,121285,"[성남시청] 9.1~9.2 함바식당(금토동 321-4), 백암한식부페(시흥동 293...",cbs
3,2021/09/07 10:59:12,68,경상남도 창원시,121284,"[정정][창원시청] 9.6. 확진자 17명 발생 ▲지역별(의창2,성산4,합포1,회원...",cbs
4,2021/09/07 10:56:25,183,전라남도 광양시,121283,[광양시청] ▶ 9.4.(토) 14:40~15:40 ▶ 미스터케이PC방 (광양읍 칠...,cbs
...,...,...,...,...,...,...
2795,2021/09/06 17:57:25,226,충청남도 당진시,121222,[당진시청]9/3(금) 18시50분경 석문국가산업단지(석문면 통정리)에서 실종 된 ...,cbs
2796,2021/09/06 17:53:50,91,경상북도 울릉군,121221,[울릉군청] 9.2.(목) 13:20~13:40 CU울릉사동점(울릉순환로 724) ...,cbs
2797,2021/09/06 17:51:09,183,전라남도 광양시,121220,[광양시청] 관내 확진자 다수 발생에 따라 금일(9.6) 보건소 선별진료소(광양읍 ...,cbs
2798,2021/09/06 17:49:31,249,충청북도 충주시,121219,"[충주시청] 9월 6일 확진자 4명 발생[자가격리 3, 타지역 확진자 접촉 1] ▲...",cbs
